# 1. kodutöö: täida lüngad

In [1]:
import os
import sys
import random
import nltk
import estnltk
from estnltk import Text
from datetime import datetime 
from estnltk.corpus_processing.parse_koondkorpus import parse_tei_corpus, parse_tei_corpora
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

C:\Users\Joosep\anaconda3\envs\Keeletehnoloogia\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Joosep\anaconda3\envs\Keeletehnoloogia\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\Joosep\anaconda3\envs\Keeletehnoloogia\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


## Andmete sisselugemine

In [2]:
def lahendaLyngad(gaps_faili_nimi, vars_faili_nimi, korpuse_asukoht):
    # lünktekst
    gap_fnm = gaps_faili_nimi
    # vastavate kandidaatide nimistu
    cnd_fnm = vars_faili_nimi
    # korpusefailide asukoht
    corp_loc = korpuse_asukoht
    # vastused
    rankings = []
    
    with open (gap_fnm, 'r', encoding='utf-8') as gap_file:
        gaps = gap_file.readlines()
    with open (cnd_fnm, 'r', encoding='utf-8') as cnd_file:
        cnds = cnd_file.readlines()
    
    #Kontrolli, kas sisendfailides on ridu sama palju
    if len(gaps)!= len(cnds):
        print("Sisendiks saadud lausete ja kandidaatide arvud ei klapi.")
        return
    
    korpuseinfo = []
    #=======================================================
    # ToDo
    #=======================================================
    korpuseinfo = loeKorpus(corp_loc)
    
    for i,gap in enumerate(gaps):
        # Loe sisse üks rida (lause) lünktekstist
        gap = gap.rstrip()
        # Löö tühikute kohalt lahkuk
        items = gap.split()
        # Kus on lünk - rea alguses olev sõne
        word_offset = int(items[0])
        sentence = items[1:]
        # lüngasõne
        gap_word = sentence[word_offset]
    
        # loe sisse üks rida (kandidaadid) teisest failist
        cnd = cnds[i].rstrip()
        candidates = cnd.split()
       
        # Lisa õige sõna kandidaatide hulka
        candidates.append(gap_word)
        # Sega kõik läbi, et oleks aus
        random.shuffle(candidates)
        # Ja võta lausest õige sõna välja
        sentence[word_offset] = ""
        
        #=========================================
        # ToDo
        candidates = jarjesta(sentence,word_offset,candidates,korpuseinfo)
        #=========================================
        
        # Vaata kõik läbi ja ütle, mitmes oli õige
        for j, c in enumerate(candidates):
            if c == gap_word:
                rankings.append(j)
                break
                
    #Vastuseid peab olema õige arv
    if len(rankings)!=len(gaps):
        print("Midagi on valesti. Sain",len(gaps),"lausele",len(rankings),"hinnangut")
    return rankings

In [3]:
def loeKorpus(korpusekaust):
    # Loeme korpuse sisse otse tokeniseeritud tekstiks
    tokenized_text = [list(map(str.lower, nltk.word_tokenize(sent))) for sent in 
                     [" ".join(artikkel.tag_layer().sentences.text) for artikkel in 
                      parse_tei_corpora(korpusekaust, suffix='.xml', target=['artikkel'])]]
            
    # Loome trigrammide tõenäosuse mugavamaks leidmiseks sobiliku mudeli
    n = 3
    train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

    model = MLE(n)
    model.fit(train_data, padded_sents)
    
    # Tagastan täisteksti ja valmis mudeli
    return model

In [4]:
def jarjesta(lyngalause, lynga_asukoht, kandidaadid, korpuseinfo):
    ###
    # Reasta kandidaadid nii, et kõige tõenäolisem oleks esimene
    # lyngalause - lause sõnede list, kus otsitava sõne asemel on tühisõne
    # lynga_asukoht - mitmes sõne lauses on lünk
    # kandidaadid - üks õige ja 200 vale vastust
    # korpuseinfo - meetodi loeKorpus väljund
    ###
    probmap = dict()
    trigramm = ['', '', '']
    
    # Iga lünga kohta peame leidma iga kandidaadi tõenäosuse
    for kandidaat in kandidaadid:
        lyngalause[lynga_asukoht] = kandidaat
        # Tõenäosustega korrutamisel algväärtus 1
        kandidaadi_prob = 1
        # Vastavas plokis katame ära kõik võimalikud trigrammid
        # mis katavad lünka, kui läheme piiridest välja, siis
        # vaatame järgmist tsükli sammu
        if len(lyngalause) > 2:
            for i in range(3):
                try:
                    trigramm[0] = lyngalause[lynga_asukoht-i]
                    trigramm[1] = lyngalause[lynga_asukoht+1-i]
                    trigramm[2] = lyngalause[lynga_asukoht+2-i]
                    # Leitud trigrami puhul korrutame kandidaadi tõenäosuse trigrami tõenäosusega
                    kandidaadi_prob*=interpoleeritudTN(trigramm, korpuseinfo)
                except:
                    continue
        # Lisame kandidaadi ja tema tõenäosuse probability mapi
        probmap[kandidaat] = kandidaadi_prob
    # Tagastame kandidaatide paremusjärjestuse sorteeritud probmapi abil
    return [k for k in dict(sorted(probmap.items(), key=lambda item: item[1], reverse=True)).keys()]

In [5]:
def interpoleeritudTN(trigramm,mudel):
    ###
    # trigramm  - nt ['kes','julgeb','ise']
    # korpuseInfo - meetodi loeKorpus väljund
    # 
    # toenaosus baaslahendusel
    # 0.85*p(ise | kes julgeb) + 0.1*p(ise | julgeb) + 0.04*p(ise) + 0.01
    # Leiame trigrammi tõenäosuse tükikaupa
    pKolm = mudel.score(trigramm[2], (trigramm[0]+" "+trigramm[1]).split())
    pKaks = mudel.score(trigramm[2], trigramm[1].split())
    pUks = mudel.score(trigramm[2])
    # Korrutame tükid vastavalt etteantud valemile
    toenaosus = 0.85*pKolm + 0.10*pKaks + 0.04*pUks + 0.01
    if toenaosus > 1:
        raise Exception("Midagi on valesti, tõenäosus on üle ühe!")
    return toenaosus

## Hindamine

In [6]:
def top10(vastused):
    # Tee top-10 jaoks alus
    # [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ranks = [0] * 10
    # Ja kui oled esikümnes, siis võta arvesse
    for rank in vastused:
        if rank < 10:
            ranks[rank] += 1
    #[6, 8, 14, 6, 5, 6, 3, 8, 9, 7]
    # Kumuleeruvalt
    for i in range(1, 10):
        ranks[i] += ranks[i-1]
    #[6, 14, 28, 34, 39, 45, 48, 56, 65, 72]
    # Kõik kokku
    n = len(vastused)
    # Suht- mitte absoluutarvuna
    acc = [r / n for r in ranks]
    print(','.join('%.2f' % a for a in acc))
    #[0.01, 0.01, 0.03, 0.03, 0.04, 0.04, 0.05, 0.06, 0.07, 0.07]
    # ehk 1% vastuseid oli esimene valik
    #     5% vastustest olid esiseitsmes
    #     (umbes 1% täpselt seitsmendal kohal)

## Testimine

In [7]:
import time
start_time = time.time()
vastused = lahendaLyngad('gaps.txt','var.txt','korpus')
print("--- %s seconds ---" % (time.time() - start_time))
koondinfo = top10(vastused)
koondinfo 


--- 542.9926500320435 seconds ---
0.12,0.16,0.18,0.20,0.22,0.24,0.25,0.26,0.28,0.29
